In [ ]:
import pandas as pd # importa a biblioteca pandas
import numpy as np #importa a biblioteca numpy

In [ ]:
dados = pd.read_json(path_or_buf='imoveis.json',orient='columns') # importa os dados

In [ ]:
dados

In [ ]:
dados.ident[0] #exibe a primeira linha



In [ ]:
dados.listing[0] #lista os dados da primeira linha

In [ ]:
# Pré processamento dos dados
#Função normalize
dados_lista1 = pd.json_normalize(dados.ident) # serve para transformar os dados em tabela
dados_lista1.head() #exibe as primeiras linhas do dataframe

In [ ]:
dados_lista2=pd.json_normalize(dados.listing,sep='_') # serve para transformar os dados em tabela
dados_lista2.head() #exibe as primeiras linhas do dataframe

In [ ]:
# Junção dos dataframes
dados_imoveis= pd.concat([dados_lista1,dados_lista2],axis=1) # junta os dois dataframes

In [ ]:
dados_imoveis.head() 

In [ ]:
dados_imoveis.shape #exibe as dimensoes do dataframe

In [ ]:
for coluna in dados_imoveis.columns: # for para printar as colunas do dataframe
    print('---- '*10)
    print(dados_imoveis[coluna].value_counts())


In [ ]:
#Pré processamento dos dados
#aplicando filtros no dataset
filtro = (dados_imoveis['types_usage']== 'Residencial' )& (dados_imoveis['address_city']== 'Rio de Janeiro') # aplicando filtro no dataseet
dados_imoveis= dados_imoveis[filtro] 
dados_imoveis.head()

In [ ]:
dados_imoveis.info() # verificando as informações dos dados

In [ ]:
# resetando index
dados_imoveis.reset_index(drop= True, inplace = True)  #reseta o index, para que as informações passem a ser ordenadas corretamente

In [ ]:
dados_imoveis.info(verbose= False) # comando para exibir somente os indices sem exibir o conteudo

In [ ]:
#transformação dos tipos de dados 
dados_imoveis = dados_imoveis.astype({

    'prices_price':'float64',
    'prices_tax_iptu':'float64',
    'prices_tax_condo':'float64',
    'features_usableAreas':'int64',
    'features_totalAreas':'int64'
}
)
dados_imoveis.info()

In [ ]:
# Lidando com dados nulos
dados_imoveis.address_zone.value_counts()

In [ ]:
dados_imoveis['address_zone']=dados_imoveis['address_zone'].replace('',np.nan) #substituição dos valores vazios por nan

In [ ]:
dados_imoveis['address_zone'].isnull().sum() # realizando a soma dos valores vazios

In [ ]:
dados_imoveis.head()

In [ ]:
dici = dados_imoveis[~dados_imoveis['address_zone'].isna()].drop_duplicates(subset=['address_neighborhood']).to_dict('records') # remove valores duplicados
print(dici)

In [ ]:
dic_zonas = {dic['address_neighborhood']:dic['address_zone']for dic in dici} #criando dicionario para associar
# bairros e zonas


In [ ]:
print(dic_zonas)

In [ ]:
#associando o bairro com a zona
for bairro, zona in dic_zonas.items(): 
    dados_imoveis.loc[dados_imoveis['address_neighborhood']==bairro,'address_zone']= zona

In [ ]:
dados_imoveis.head() # exibindo as primeiras linhas

In [ ]:
dados_imoveis['address_zone'].isnull().sum() # verificando se ainda tem valores faltantes

In [ ]:
#verificando se tem outros valores faltantes nas outras colunas
dados_imoveis.prices_tax_condo.isnull().sum()

In [ ]:
#verificando se tem outros valores faltantes nas outras colunas
dados_imoveis.prices_tax_iptu.isnull().sum()

In [ ]:
#Corrigindo os valores faltantes
dados_imoveis.prices_tax_iptu.fillna(0,inplace = True) # função para substituir os valores vazios
dados_imoveis.prices_tax_condo.fillna(0,inplace = True) #função para substituir os valores vazios
print(f"Total de prices nulos: {dados_imoveis.prices_tax_iptu.isnull().sum()}")
print(f"Total de prices nulos: {dados_imoveis.prices_tax_condo.isnull().sum()}")

In [ ]:
#Remove colunas nao uteis para criar o modelo
dados_imoveis.drop(['customerID','source','types_usage','address_city',
                    'address_location_lon','address_location_lat','address_neighborhood'],axis=1, inplace= True)

In [ ]:
#criando dicionario para renomear as colunas
dic_colunas={
    'types_unit':'unit','address_zone': 'zone','prices_price':'price',
    'prices_tax_condo':'tax_condo','prices_tax_iptu':'tax_iptu','features_bedrooms':'bedrooms',
    'features_bathrooms':'bathrooms','features_suites':'suites','features_parkingSpaces':'parkingSpaces',
    'features_usableAreas':'usableAreas', 'features_totalAreas':'totalAreas', 'features_floors':'floors',
    'features_unitsOnTheFloor': 'unitsOnTheFloor','features_unitFloor':'unitFloor'
}

In [ ]:
#renomeando as colunas
dados_imoveis= dados_imoveis.rename(dic_colunas, axis =1) #renomeia as colunas conforme o dicionario criado

In [ ]:
dados_imoveis.head()

In [ ]:
col_n = dados_imoveis.select_dtypes(include=['number']) # selecionando apenas colunas com numeros



In [ ]:
correlacao= col_n.corr()

In [ ]:
correlacao

In [ ]:
pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#Visualização das correlações
cores = sns.color_palette('light:salmon', as_cmap = True) # personalização das cores
mask = np.zeros_like(correlacao)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style('white'):
    f,ax= plt.subplots(figsize=(13,8))
    ax = sns.heatmap(correlacao, cmap=cores, mask = mask, square = True, fmt = '.2f',annot = True)

In [ ]:
sns.heatmap(correlacao, cmap='crest')

In [ ]:
plt.figure(figsize=(13,8))
mask = np.zeros_like(correlacao)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(correlacao, cmap='crest', mask = mask, square = True, fmt = '.2f',annot = True)

In [ ]:
ax = sns.histplot(data = dados_imoveis, x ='price', kde= True)
ax.figure.set_size_inches(20,10)
ax.set_title('Histograma de preços')
ax.set_xlabel('Preços')
ax.set_xlabel('Preço');

Transformação das variáveis
Estamos realizando uma transformação logarítmica das variáveis para obtermos uma distribuição simétrica (mais próxima da normal), para que possamos usar o modelo de regressão linear. A transformação realizada é:

Lembrar de realizar a transformação inversa para obter o valor original:

Para mais detalhes consultar as documentações log1p, expm1 e FunctionTransformer

In [ ]:
from sklearn.preprocessing import FunctionTransformer #importar metodo para realizar a transformação dos dados

In [ ]:
transformer = FunctionTransformer(np.log1p, validate=True)
dados_transformados = transformer.transform(dados_imoveis.select_dtypes(exclude=['object'])) #exclui dados que não sao numéricos

In [ ]:
colunas_dados_tranformados = dados_imoveis.select_dtypes(exclude=['object']).columns

df_transformado = pd.concat([dados_imoveis.select_dtypes(include=['object']), pd.DataFrame(dados_transformados, columns=colunas_dados_tranformados)], axis=1)
df_transformado.head()

In [ ]:
#depois da transformação
col_n = dados_imoveis.select_dtypes(include=['number']) # selecionando apenas colunas com numeros
correlacao_transformado = col_n.corr()
mask = np.zeros_like(correlacao_transformado)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(13, 8))
    ax = sns.heatmap(correlacao_transformado, cmap=cores, mask=mask, square=True, fmt='.2f', annot=True)
    ax.set_title('Correlação entre variáveis - Tranformação Log', fontsize=20)

In [ ]:
#depois (distrib. simétrica)
ax = sns.histplot(data=df_transformado, x='price', kde=True)
ax.figure.set_size_inches(20, 10)
ax.set_title('Histograma de preços')
ax.set_xlabel('Preço');

In [ ]:
variaveis_categoricas = df_transformado.select_dtypes(include=['object']).columns #variaveis categóricas

In [ ]:
df_dummies = pd.get_dummies(df_transformado[variaveis_categoricas]) # cria uma nova coluna com variaveis categóricas
df_dummies.head()

In [ ]:
dados_imoveis_dummies = pd.concat([df_transformado.drop(variaveis_categoricas, axis=1), df_dummies], axis=1) 
dados_imoveis_dummies.head()

Ajustes e previsão

In [ ]:
#Ajuste e previsao
#variáveis explanatórias (independentes)
X = dados_imoveis_dummies.drop('price', axis=1)

#variável dependente / Variavel dependente
y = dados_imoveis_dummies['price']

In [ ]:
from sklearn.model_selection import train_test_split #Dividir o conjunto de dados para treino e teste

In [ ]:
#divisão em conjunto de treino e teste
#random_state - estado de aleatoriedade
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42) #função para dividir o conjunto de dados

In [ ]:
from sklearn.linear_model import LinearRegression # importa o modelo de regressao linear


In [ ]:
#Instanciando o modelo
lr = LinearRegression()

In [ ]:
#treino
lr.fit(X_treino, y_treino) #treino do modelo

In [ ]:
#teste
previsao_lr = lr.predict(X_teste)

In [ ]:
X_teste

In [ ]:
np.expm1(7.49)

In [ ]:
dados_imoveis.head(5)

In [ ]:
#resultados da previsão
previsao_lr

In [ ]:
np.expm1(12.45)

In [ ]:
np.expm1(13.13161073)

In [ ]:
#transformação inversa 
np.expm1(13.25768024)

2.4 - Métrica de desempenho
Coeficiente de Determinação
descobrir quão “bem” uma linha de regressão amostral é adequada aos dados
r2 situa-se entre 0 e 1
"r2 mede a proporção ou percentual da variação total de Y explicada pelo modelo de regressão."

In [ ]:
#importar a biblioteca para calcular a métrica r2_score
from sklearn.metrics import r2_score 

In [ ]:
r2_lr = r2_score(y_teste, previsao_lr)
r2_lr

Extra
Documentação Sklearn - Métricas para Regressão

Observação: Estas métricas dependem da escala dos dados!!!

MSE (Mean Squared Error ou Erro Quadrático Médio): média da diferença elevada ao quadrado entre o valor real e o previsto. (penalidade sobre o erro)

MAE (Mean Absolute Error ou Erro Absoluto Médio): média da diferença absoluta (módulo) entre o valor real e o previsto.

In [ ]:
from sklearn.metrics import r2_score
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
r2_score(y_true, y_pred)

In [ ]:
from sklearn.metrics import mean_absolute_error

y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_absolute_error(y_true, y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error

y_true = [[0.5, 1],[-1, 1],[7, -6]]
y_pred = [[0, 2],[-1, 2],[8, -5]]

mean_squared_error(y_true, y_pred, squared=True)


In [ ]:
from sklearn.metrics import mean_squared_error

y_true = [[0.5, 1],[-1, 1],[7, -6]]
y_pred = [[0, 2],[-1, 2],[8, -5]]

mean_squared_error(y_true, y_pred, squared=False)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]

mean_absolute_percentage_error(y_true, y_pred)


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_teste, previsao_lr)
mape

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_teste, previsao_lr)
mse

In [ ]:
from sklearn.metrics import mean_squared_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_squared_error(y_true, y_pred)


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_absolute_percentage_error(y_true, y_pred)

In [ ]:
#arvore de decisão para regressao
from sklearn.tree import DecisionTreeRegressor

In [ ]:
#Instanciando o modelo
dtr = DecisionTreeRegressor(random_state=42,max_depth=5) #random_state garante que a estrutura da árvore será reprodutivel
#max_depth define o tamanho da arvore


In [ ]:
#treino
dtr.fit(X_treino,y_treino)

In [ ]:
#teste
previsao_dtr = dtr.predict(X_teste)

In [ ]:
previsao_dtr

In [ ]:
np.expm1(13.55136531)

In [ ]:
pip install yellowbrick

In [ ]:
from yellowbrick.regressor import PredictionError

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
pev = PredictionError(dtr)
pev.fit(X_treino,y_treino)
pev.score(X_teste,y_teste)
pev.poof()

In [ ]:
#Metrica
r2_dtr = r2_score(y_teste,previsao_dtr)
r2_dtr

Parâmetros Decision Tree Regressor
n_estimators: quantidade de arvores na floresta
bootstrap: define se será utilizado o metódo estatístico de amostragem bootstrap
oob_Score: define se será utilizado o método de amostragem out of bag para mensurar os erros da predição
n_jobs: número de jobs rodando paralelamente (computação paralela - processadores)
warm_start: faz a adequação de uma nova floresta ou usa uma floresta existente
max_samples: para o caso do bootstrap ser True, define o número de amostras a serem extraídas de X, para treinar
cada arvore base

In [ ]:
#Metódo Ensemble
#Random Forest
from sklearn.ensemble import RandomForestRegressor


In [ ]:
rf = RandomForestRegressor(random_state=42, max_depth=5, n_estimators=10)

In [ ]:
rf.fit(X_treino,y_treino)

In [ ]:
previsao_rf = rf.predict(X_teste)

In [ ]:
previsao_rf

In [ ]:
np.expm1(13.53707348)

In [ ]:
from yellowbrick.regressor import PredictionError

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
pev = PredictionError(rf)
pev.fit(X_treino,y_treino)
pev.score(X_teste, y_teste)
pev.poof()

In [ ]:
#Metricas de desempenho
r2_rf = r2_score(y_teste,previsao_rf)
r2_rf

In [ ]:
metricas_modelo_ML = pd.DataFrame({
    'Modelo': ['Regressao Linear', 'Árvore de decisão','Random Forest'],
     'Metricas': ['R2','MSE','MAE']
})

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error

In [ ]:
def obter_metrica(modelo, X_teste, y_teste, nome):
    predict = modelo.predict(X_teste)
    df_metricas = pd.DataFrame({
        'R2':[r2_score(y_teste,predict)],
        'MSE':[mean_squared_error(y_teste,predict)],
        'MAE':[mean_absolute_error(y_teste,predict)]
    },index=[nome])
    return df_metricas

In [ ]:
def tabela_metricas(modelo_reg_linear, modelo_dt, modelo_rf, X_teste, y_teste):
    df_metricas_reg_linear = obter_metrica(modelo_reg_linear, X_teste, y_teste, 'Linear Regression')
    df_metricas_dt = obter_metrica(modelo_dt, X_teste, y_teste, 'Decision Tree Regression')
    df_metricas_rf = obter_metrica(modelo_rf, X_teste, y_teste, 'Random Forest Regression')

    return pd.concat([df_metricas_reg_linear, df_metricas_dt, df_metricas_rf])

In [ ]:
tabela_metricas(lr, dtr, rf, X_teste, y_teste)

In [ ]:
metricas_modelo_ML.head()